<a href="https://colab.research.google.com/github/gp01gp02gp03155-hash/1-dim-Go-program-with-DMP/blob/main/1dimGo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

1次元囲碁の盤面を3xNの行列で表現し、指定されたルール（空きマスへの着手、相手の石の除去、相手の石を取れない自殺手の禁止、パス）を実装したゲームコード全体です。

In [43]:
import numpy as np

def board_to_matrix(board):
    """
    1次元盤面を3xNの行列に変換します。
    行列の行はそれぞれ '.', 'B', 'W' に対応します。
    """
    matrix = np.zeros((3, len(board)), dtype=int)
    for i, cell in enumerate(board):
        if cell == '.':
            matrix[0, i] = 1
        elif cell == 'B':
            matrix[1, i] = 1
        elif cell == 'W':
            matrix[2, i] = 1
    return matrix

def matrix_to_board(matrix):
    """
    3xNの行列を1次元盤面に変換します。
    """
    board = []
    for i in range(matrix.shape[1]):
        if matrix[0, i] == 1:
            board.append('.')
        elif matrix[1, i] == 1:
            board.append('B')
        elif matrix[2, i] == 1:
            board.append('W')
        else:
            board.append('?') # 不明な状態
    return board

def get_opponent(player):
    """
    プレイヤーの相手を取得します。
    """
    return 'W' if player == 'B' else 'B'

def remove_captured_matrix(matrix, player):
    """
    行列表現された盤面から、指定されたプレイヤーによって取られた相手の石を除去します。
    取った石の数を返します。
    """
    opponent = get_opponent(player)
    opponent_row = 1 if opponent == 'B' else 2
    player_row = 1 if player == 'B' else 2
    board_size = matrix.shape[1]
    updated_matrix = matrix.copy()
    captured_indices = []

    # 1次元なので、挟まれているかを確認
    for i in range(board_size):
        if updated_matrix[opponent_row, i] == 1: # 相手の石があるか
            # 左端の相手の石が右隣の自分の石に取られるケース
            if i == 0 and board_size > 1 and updated_matrix[player_row, i+1] == 1:
                 captured_indices.append(i)
            # 右端の相手の石が左隣の自分の石に取られるケース
            elif i == board_size - 1 and board_size > 1 and updated_matrix[player_row, i-1] == 1:
                 captured_indices.append(i)
            # 中央の相手の石が両隣の自分の石に挟まれて取られるケース
            elif i > 0 and i < board_size - 1 and updated_matrix[player_row, i-1] == 1 and updated_matrix[player_row, i+1] == 1:
                captured_indices.append(i)

    # 挟まれた石を空きセルに戻す
    for idx in captured_indices:
        updated_matrix[opponent_row, idx] = 0
        updated_matrix[0, idx] = 1 # 空きセルにする

    return updated_matrix, len(captured_indices) # 更新された行列と取った石の数を返す

# ヘルパー関数：指定された位置の石を含む連（string）とその呼吸点を取得（1次元用）
def get_string_and_liberties(matrix, pos, current_player):
    player_row = 1 if current_player == 'B' else 2
    # opponent_row = 1 if get_opponent(current_player) == 'B' else 2 # This variable is not used
    board_size = matrix.shape[1]

    if matrix[player_row, pos] == 0: # 指定された位置に自分の石がない
        return [], set()

    string = set()
    liberties = set()
    q = [pos]
    visited = {pos}

    while q:
        curr = q.pop(0)
        string.add(curr)

        # 隣接する位置をチェック (左と右)
        neighbors = []
        if curr > 0:
            neighbors.append(curr - 1)
        if curr < board_size - 1:
            neighbors.append(curr + 1)

        for neighbor in neighbors:
            if matrix[0, neighbor] == 1: # 空きマスは呼吸点
                liberties.add(neighbor)
            elif matrix[player_row, neighbor] == 1 and neighbor not in visited: # 隣接する自分の石は同じ連
                visited.add(neighbor)
                q.append(neighbor)
            # 相手の石の場合は呼吸点ではない (呼吸点ではないのでここでは何もしない)

    # 隣接する自分の石の連を再帰的に探索して呼吸点を集める
    # この1次元のケースでは、隣接する自分の石は常に同じ連とみなせるため、
    # 上記のwhileループで連全体とそれに直接隣接する呼吸点（空きマス）は取得できています。
    # したがって、再帰的な探索は不要です。

    return sorted(list(string)), liberties


def is_legal_move(matrix, pos, player):
    """
    指定されたプレイヤーが指定された位置に石を置くのが合法手であるかを判定します。
    （相手の石を取れない自殺手は禁止）
    """
    board_size = matrix.shape[1]
    # print(f"Checking legal move for {player} at position {pos}") # Debug print - Comment out for cleaner output

    # 1. 位置が有効範囲内か
    if pos < 0 or pos >= board_size:
        # print(f"  Invalid position: {pos}") # Debug print - Comment out for cleaner output
        return False

    # 2. 空きマスか
    if matrix[0, pos] == 0:
        # print(f"  Position {pos} is not empty.") # Debug print - Comment out for cleaner output
        return False

    # 3. 石を置いた場合に自殺手にならないか（ただし石を取れる場合は合法）
    # その位置に石を置いた場合の仮想的な盤面を作成
    temp_matrix = matrix.copy()
    player_row = 1 if player == 'B' else 2
    temp_matrix[0, pos] = 0 # 空きをなくす
    temp_matrix[player_row, pos] = 1 # 自分の石を仮置き

    # print(f"  Temp matrix after placing stone:\n{temp_matrix}") # Debug print - Comment out for cleaner output

    # 仮想的な盤面で相手の石が取られるかを確認
    matrix_after_potential_capture, captured_count = remove_captured_matrix(temp_matrix, player) # 取った石の数も受け取る
    # print(f"  Matrix after potential capture:\n{matrix_after_potential_capture}") # Debug print - Comment out for cleaner output


    # 石が取られたかどうかの判定（行列を比較）
    if captured_count > 0: # 石が取れた場合
        # 石が取れる場合は合法手
        # print(f"  Move at {pos} captures opponent stone(s). Legal move.") # Debug print - Comment out for cleaner output
        return True
    else:
        # 石が取れなかった場合、自殺手でないか確認
        # 新しく置いた石を含む連とその呼吸点を取得
        string, liberties = get_string_and_liberties(temp_matrix, pos, player)
        # print(f"  Move at {pos} does not capture. Checking suicide.") # Debug print - Comment out for cleaner output
        # print(f"  String at {pos}: {string}, Liberties: {liberties}") # Debug print - Comment out for cleaner output


        # 呼吸点が0の場合は自殺手 -> 非合法手
        if not liberties:
            # print(f"  Move at {pos} is suicide (0 liberties). Illegal move.") # Debug print - Comment out for cleaner output
            return False
        else:
            # print(f"  Move at {pos} is not suicide ({len(liberties)} liberties). Legal move.") # Debug print - Comment out for cleaner output
            return True


def main_matrix_game():
    """
    行列表現を用いた1次元囲碁のゲーム実行関数。
    石の配置、除去、合法手がない場合の自動パス、連続パスによるゲーム終了ルールを含みます。
    ゲーム終了時にスコア（盤上の石 + 取った石）を表示します。
    """
    while True:
        try:
            BOARD_SIZE_input = input("盤面のサイズ (N) を入力してください (終了する場合は 'q'): ")
            if BOARD_SIZE_input.lower() == 'q':
                print("ゲームを終了します。")
                return # ゲーム終了

            BOARD_SIZE = int(BOARD_SIZE_input)
            if BOARD_SIZE <= 0:
                print("盤面サイズは1以上の整数を入力してください。")
                continue
            break # 有効な入力があればループを抜ける
        except ValueError:
            print("無効な入力です。整数を入力してください。")


    board_matrix = board_to_matrix(['.'] * BOARD_SIZE)
    player = 'B'
    consecutive_passes = 0 # 連続パス回数を記録
    b_captured_stones = 0 # Bが取った石の数
    w_captured_stones = 0 # Wが取った石の数


    print(f"1次元囲碁 ({BOARD_SIZE}x1盤面, 行列表現)")

    while True:
        print("\n現在の盤面:")
        print(matrix_to_board(board_matrix))

        # 合法手の確認
        legal_moves = [pos for pos in range(BOARD_SIZE) if is_legal_move(board_matrix, pos, player)]

        if not legal_moves:
            print(f"{player} は合法手がありません。自動的にパスします。")
            move = 'p' # 自動パス
        else:
             print(f"{player}'s turn. Legal moves: {legal_moves}")
             move = input(f"{player}'s move ({'/'.join(map(str, legal_moves))}, 'p' to pass, 'q' to quit): ")


        if move.lower() == 'q':
            break
        elif move.lower() == 'p':
            print(f"{player} がパスしました。")
            consecutive_passes += 1

            # 連続パスが2回になったらゲーム終了
            if consecutive_passes >= 2:
                 print("両プレイヤーが連続でパスしたためゲーム終了。")
                 break

            player = get_opponent(player) # パスしたら相手のターン
            continue # 次のループへ

        # 数字入力の場合
        try:
            pos = int(move)
            # 入力された手が合法手リストに含まれているか再確認（不正な入力を防ぐ）
            if pos in legal_moves:
                # 石を置く処理 (is_legal_moveで合法性は確認済み)

                # その位置に石を置いた場合の仮想的な盤面を作成
                temp_matrix = board_matrix.copy()
                player_row = 1 if player == 'B' else 2
                temp_matrix[0, pos] = 0 # 空きをなくす
                temp_matrix[player_row, pos] = 1 # 自分の石を仮置き

                # 仮想的な盤面で相手の石が取られるかを確認し、取った石の数も取得
                matrix_after_potential_capture, captured_count = remove_captured_matrix(temp_matrix, player)

                # 石が取られたかどうかの判定（取った石の数で判断）
                if captured_count > 0:
                     # 石が取れる場合は、石を取った状態を反映
                     board_matrix = matrix_after_potential_capture
                     print(f"相手の石を {captured_count} 個取りました。")
                     if player == 'B':
                         b_captured_stones += captured_count
                     else:
                         w_captured_stones += captured_count
                else:
                     # 石が取れない場合は、石を置いた状態を反映
                     board_matrix = temp_matrix

                # 石を置いた後は連続パスをリセット
                consecutive_passes = 0

                player = get_opponent(player) # ターン交代

            else:
                print(f"無効な入力です。合法手 ({'/'.join(map(str, legal_moves))})、'p' または 'q' を入力してください。")
                # 無効な手の場合は連続パスをリセットしない（連続パスのカウントは有効な手でのみリセット）
                continue # 同じプレイヤーのターンを維持


        except ValueError:
            print(f"無効な入力です。合法手 ({'/'.join(map(str, legal_moves))})、'p' または 'q' を入力してください。")
            # 無効な手の場合は連続パスをリセットしない
            continue # 同じプレイヤーのターンを維持


    print("\nゲーム終了")
    print("最終盤面:")
    print(matrix_to_board(board_matrix))

    # 最終スコアの計算と表示
    b_board_stones = np.sum(board_matrix[1, :])
    w_board_stones = np.sum(board_matrix[2, :])

    b_total_score = b_board_stones + b_captured_stones
    w_total_score = w_board_stones + w_captured_stones

    print("\n--- 最終スコア ---")
    print(f"Bのスコア: 盤面の石 ({b_board_stones}) + 取った石 ({b_captured_stones}) = {b_total_score}")
    print(f"Wのスコア: 盤面の石 ({w_board_stones}) + 取った石 ({w_captured_stones}) = {w_total_score}")
    print("------------------")


# ゲームを開始する場合は以下の行のコメントを外してください
# main_matrix_game()

In [44]:
main_matrix_game()

盤面のサイズ (N) を入力してください (終了する場合は 'q'): 5
1次元囲碁 (5x1盤面, 行列表現)

現在の盤面:
['.', '.', '.', '.', '.']
B's turn. Legal moves: [0, 1, 2, 3, 4]
B's move (0/1/2/3/4, 'p' to pass, 'q' to quit): 0

現在の盤面:
['B', '.', '.', '.', '.']
W's turn. Legal moves: [1, 2, 3, 4]
W's move (1/2/3/4, 'p' to pass, 'q' to quit): 1
相手の石を 1 個取りました。

現在の盤面:
['.', 'W', '.', '.', '.']
B's turn. Legal moves: [2, 3, 4]
B's move (2/3/4, 'p' to pass, 'q' to quit): 2

現在の盤面:
['.', 'W', 'B', '.', '.']
W's turn. Legal moves: [3, 4]
W's move (3/4, 'p' to pass, 'q' to quit): 3
相手の石を 1 個取りました。

現在の盤面:
['.', 'W', '.', 'W', '.']
B は合法手がありません。自動的にパスします。
B がパスしました。

現在の盤面:
['.', 'W', '.', 'W', '.']
W's turn. Legal moves: [0, 2, 4]
W's move (0/2/4, 'p' to pass, 'q' to quit): 4

現在の盤面:
['.', 'W', '.', 'W', 'W']
B は合法手がありません。自動的にパスします。
B がパスしました。

現在の盤面:
['.', 'W', '.', 'W', 'W']
W's turn. Legal moves: [0, 2]
W's move (0/2, 'p' to pass, 'q' to quit): 0

現在の盤面:
['W', 'W', '.', 'W', 'W']
B は合法手がありません。自動的にパスします。
B がパスしました。

現在の盤面:
['W', 'W', '